# Lista 9

In [1]:

import cv2
import numpy as np

In [3]:
video_path = 'onca.mp4'
# 8 frames inseridos a cada par de frames consecutivos do vídeo original
fator = 8 

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [4]:
#  criar os vídeos de output
# interpolacao por repeticao
outrep_width = width
outrep_height = height
# interpolacao linear
outlin_width = width
outlin_height = height
# interpolacao por fluxo optico
outopt_width = width
outopt_height = height
# video com os 3 métodos combinados lado a lado
outcomb_width = 3*width 
outcomb_height = height

# Criar objeto VideoWriter para salvar os vídeos
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
outrep_path = 'out_rep.mp4'
outlin_path = 'out_lin.mp4'
outopt_path = 'out_opt.mp4'
outcomb_path = 'out_comb.mp4'

outrep = cv2.VideoWriter(outrep_path, fourcc, fps, (outrep_width, outrep_height))
outlin = cv2.VideoWriter(outlin_path, fourcc, fps, (outlin_width, outlin_height))
outopt = cv2.VideoWriter(outopt_path, fourcc, fps, (outopt_width, outopt_height))
outcomb = cv2.VideoWriter(outcomb_path, fourcc, fps, (outcomb_width, outcomb_height))

In [5]:
# Função auxiliar para combinar os frames dos vídeos em um único frame
def combinar_frames(frames):
    # Define as dimensões do novo frame
    height = frames[0][0].shape[0]
    width = frames[0][0].shape[1]
    channels = frames[0][0].shape[2]
    combined_frame = np.zeros((height, width * len(frames), channels), dtype=np.uint8)

    # Combina os frames dos vídeos
    for i, frame in enumerate(frames):
        combined_frame[:, i * width : (i + 1) * width, :] = frame[0]

    return combined_frame

In [7]:

# mapa das coordenadas x e y (video original) - para o método por fluxo ótico
coord_x, coord_y = np.meshgrid(np.arange(width), np.arange(height))

cont_frames = 0; total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT); bloco = int(total_frames/10)

#rebobinar o vídeo original para o início
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, prev_frame = cap.read()
while cap.isOpened():
    cont_frames += 1

    #imprimir o progresso do processamento
    if cont_frames % bloco== 0:
        print('Processando: ', int(cont_frames/bloco)*10, '%')
    ret, frame = cap.read()

    if not ret:
        break
    # sequencia comeca com o frame anterior (prev_frame)
    frame_repeat = cv2.resize(prev_frame, (outrep_width, outrep_height))
    frame_linear = cv2.resize(prev_frame, (outlin_width, outlin_height))
    frame_optflow = cv2.resize(prev_frame, (outopt_width, outopt_height))

    frame_combinado = combinar_frames([[frame_repeat], [frame_linear], [frame_optflow]])
    frame_combinado = cv2.resize(frame_combinado, (outcomb_width, outcomb_height))
    # escreve cada frame no video de saida correspondente
    outrep.write(frame_repeat)
    outlin.write(frame_linear)
    outopt.write(frame_optflow)
    outcomb.write(frame_combinado)
    
    # Efetuar o fluxo ótico
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calcular o fluxo ótico.  Use o método de Farneback já implementado no OpenCV
    # os parâmetros do método são:
    # prev_gray: frame anterior em escala de cinza
    # gray: frame atual em escala de cinza
    # None: sem máscara
    # 0.5: pirâmide de escala, fator de escala
    # 3: número de níveis da pirâmide
    # 15: tamanho da janela de vizinhança
    # 3: número de iterações do algoritmo
    # 5: tamanho da janela de média para suavização
    # 1.2: desvio padrão do filtro gaussiano
    # 0: flags
    
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # inserir frames intermediários
    for i in range(1, fator):
        # Interpolação por repetição
        frame_repeat = prev_frame

        # Interpolação linear
        # a função addWeighted() do OpenCV pode ser usada para combinar dois frames
        # por interpolação linear.  Para combinar frames é melhor usar a função, que evita
        # que você se preocupe com o tipo numérico e problemas de arredondamento
        frame_linear = cv2.addWeighted(prev_frame, (fator - i) / fator,frame, i / fator, 0)

        # Interpolação por fluxo ótico
        # modificando as coordenadas x e y de acordo com o fluxo ótico
        # o mapa map_x contém as coordenadas x de cada pixel do frame atual
        # ele deve ser calculado a partir do mapa de coordenadas x original (coord_x)
        # e do fluxo ótico (flow)
        # use uma interpolação linear, com peso (i/fator), para incluir a fração do fluxo ótico que deve ser considerada
        # não é preciso usar a função cv2.addWeighted() aqui!  São apenas matrizes de números reais.
        map_x = coord_x #complete aqui# 
        map_y = coord_y #complete aqui#
        frame_optflow = cv2.remap(prev_frame, map_x.astype(np.float32),\
                        map_y.astype(np.float32), interpolation=cv2.INTER_LINEAR)
        # finalmente, faça a interpolação linear entre o próximo frame e o
        # frame transformado com fluxo ótico.
        # quais os pesos você deve usar?
        peso_fluxo_otico = (fator - i)/fator
        peso_proximo_frame = 1-peso_fluxo_otico
        frame_optflow = cv2.addWeighted(frame_optflow, peso_fluxo_otico, frame, peso_proximo_frame, 0)

        # Combina os frames dos vídeos (repetição, linear e fluxo ótico)
        frame_combinado = combinar_frames([[frame_repeat], [frame_linear], [frame_optflow]])
        #frame_combinado = cv2.resize(frame_combinado, (outcomb_width, outcomb_height))
        # escreve cada frame no video de saida correspondente
        outrep.write(frame_repeat)
        outlin.write(frame_linear)
        outopt.write(frame_optflow)
        outcomb.write(frame_combinado)

    prev_frame = frame

cap.release()
outrep.release()
outlin.release()
outopt.release()
outcomb.release()

Processando:  10 %
Processando:  20 %
Processando:  30 %
Processando:  40 %
Processando:  50 %
Processando:  60 %
Processando:  70 %
Processando:  80 %
Processando:  90 %
Processando:  100 %


# 2

- Frames com interpolação por repetição:
Conforme é possível verificar no resultado, existem transições bruscas entre os frames, causado pela repetição do frame anterior que é copiado.

- Frames com interpolação linear:
A transição é mais suave devido ao efeito de média utilizado para interpolar o frame intermediária, mas o movimeento ao fundo da cena ainda não apresenta uma transição muito suave.

- Frames com interpolação de fluxo ótico:
Apresenta a melhor transição, pois, levando em conta a trajetória dos objetos, o frames intermeediários preservam etalahes do movimento, e transições bruscas e borrões são quase imperceptíveis.


# 3

a)

O fluxo ótico dos pontos de destaque, para cada pixel, é dado pela seguinte equação:

$$
F = \frac{v_{cam} d_f}{60d}
$$

onde:

$v_{cam}$: velocidade da câmera (pixeis/s)

$d_f$: distância focal (pixeis)

$d$ distância entre o Objeto e a câmera

Como a câmera gera um vídeo de $60$ frames por segundo, sua velocidade (dada em pixeis/s) exige a divisão por $60$.

Assim:

$$d = \frac{v_{cam} d_f}{60 F}$$




b)

De acordo com a fórmula do item anterior, à medida que $d\to \infty$, $F\to 0$, de forma que quanto maior a distância, menor será o fluxo ótico. Esse resultado corresponde ao esperado: intuitivamente, se a distância entre eo objeto e a câmera cresce, o fluxo ótico se torna menor.

# 4

In [12]:
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import matplotlib.pyplot as plt

# Load the video
cap = cv2.VideoCapture('gatinho.mp4')
ret, first_frame = cap.read()
if not ret:
    print("Failed to load video.")
    cap.release()
    exit()

# Convert the first frame to grayscale
gray_first = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# Define the point to be tracked
point = []

def select_point(event, x, y, flags, param):
    global point
    if event == cv2.EVENT_LBUTTONDOWN:
        point = [(x, y)]
        cv2.circle(first_frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("First Frame", first_frame)

cv2.namedWindow("First Frame")
cv2.setMouseCallback("First Frame", select_point)

while True:
    cv2.imshow("First Frame", first_frame)
    if len(point) > 0:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyWindow("First Frame")

# Parameters for lucas kanade optical flow
lk_params = dict(winSize  = (15, 15),
                 maxLevel = 2,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create a mask image for drawing purposes
mask = np.zeros_like(first_frame)

# Convert point to numpy array
p0 = np.array([point], dtype=np.float32)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 30.0, (first_frame.shape[1], first_frame.shape[0]))

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(gray_first, gray_frame, p0, None, **lk_params)
    
    if st[0][0] == 1:
        a, b = p1.ravel()
        a, b = int(a), int(b)  # Convert to integers
        cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)
        mask = cv2.line(mask, tuple(p0.ravel().astype(int)), (a, b), (0, 255, 0), 2)
        frame = cv2.add(frame, mask)
        p0 = p1
        gray_first = gray_frame.copy()

    out.write(frame)
    if frame_count % 20 == 0:  # 
        _, img = cv2.imencode('.jpg', frame)
        final_image = 'final_frame.jpg'
        cv2.imwrite(final_image, frame)
        display(Image(data=img))
        clear_output(wait=True)
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 

: 